If you changed the CBSA DATABASE on the coverage study template...

YOU MUST UPDATE SHEET2 IN THE GYM DB

If the CBSA Database is not updated. The distance study may not accurately reflect the metro areas

In [52]:
%%time


import pandas as pd
import numpy as np
from geopy.distance import great_circle 
import time
import os


df = pd.read_csv('Penn Med Lat Long.csv')

gymdf = pd.read_excel('2.4.19 GYM DB PYTHON.xlsx',sheet_name="Sheet1")

# IF YOU CHANGED THE CBSA DATABASE... YOU MUST UPDATE SHEET2 IN THE GYM DB
msadf = pd.read_excel('2.4.19 GYM DB PYTHON.xlsx',sheet_name='Sheet2')

company_name = "INSERT FULL COMPANY NAME HERE"

Wall time: 12.7 s


^ Imports important libraries. 
(df: data frame) Imports Excel File with company zips, Lats, Longs, and FTE's. 
gymdf imports Gym Database (Sheet1). 
msadf imports the CBSA Database from the Gym Database file (Sheet2).

IMPORTANT: Your excel files should (xlsx supported) be in the same folder as your Distance Calculator file (ipynb). Excel column names should be "latitude" / "longitude" exactly (no uppercase letters and no preceding or trailing spaces).

In [53]:
id_list = []
title_list = []
status_list = []
city_list = []
state_list = []
Gym_Type_list = []
Pricing_Tier_list = []

dist_list = []
dist_less_than_5 = []
zip_list = []
fte_list = []
msa_list = []
emp_city_list = []

dist = 10
dist = dist * 1.6340934
latmod = dist/50
longmod = dist/60

for y in df.itertuples():
    print(". ",end="")
    i = y[0]
    templat = y[2]
    templong = y[3]
    latmin = templat - latmod
    latmax = templat + latmod
    longmax = templong + longmod 
    longmin = templong - longmod
    
    df_temp = gymdf[(gymdf["latitude"] >latmin) & (gymdf["latitude"]<latmax) & (gymdf["longitude"]>longmin) & (gymdf["longitude"]< longmax)]
    
    for z in df_temp.itertuples():
        if great_circle((y[2], y[3]),(z[10],z[11])) <= dist:
            id_list.append(z[1])
            distance = great_circle((y[2], y[3]),(z[10],z[11])).miles
            dist_list.append(distance)
            if distance < 5:
                dist_less_than_5.append("Y")
            else:
                dist_less_than_5.append("N")
            zip_list.append(y[1])
            fte_list.append(y[4])
            title_list.append(z[2])
            status_list.append(z[3])
            city_list.append(z[6])
            state_list.append(z[7])
            Gym_Type_list.append(z[8])
            Pricing_Tier_list.append(z[9])
            
            try:
                temp_msa = msadf[msadf["ZIP CODE"] == y[1]]
                msa_list.append(temp_msa['CBSA Combined'].values[0])
                emp_city_list.append(temp_msa['City'].values[0])
            except:
                msa_list.append("")
                emp_city_list.append("")
            
print("DONE")

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . DONE


^ Asks for 4 lists of data (Gym ID, Dist Calc, Company Postal Code, # of FTEs). 

BOX CREATION: In order to reduce computational demand, a temporary data frame nees to be created. For each Company Zip code, we create a slice of all the gyms that could be within 5 miles of the Company Zip. A distance of 5 miles chosen, and converted to miles. Lat is divided by 50, and Long is divided by 60. 

Row (ZIPS) is replaced with var name "y". Lat is in column 2 while Long is in column 3. Gym (Zips) is replaced with var name "z". Lat is in column 10 while Long is in column 11.

In [54]:
export = pd.DataFrame()

export['gym_id'] = id_list
export['title'] = title_list
export['status'] = status_list
export['dist'] = dist_list
export['dist < 5'] = dist_less_than_5
export['zip'] = zip_list
export['FTE'] = fte_list
export['city'] = city_list
export['state'] = state_list
export['Gym_Type'] = Gym_Type_list
export['Gym_Tier'] = Pricing_Tier_list
export['Region'] = msa_list
export['Employee City'] = emp_city_list

In [55]:
file_name = company_name + " Distance Study.xlsx"
export.to_excel(file_name, sheet_name='Raw')

^ Exports data to Excel